PAP

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np 
import warnings
warnings.filterwarnings('ignore')

from AA import DataDownloader, AssetAllocation

C:\Users\pmonr\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\pmonr\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
downloader = DataDownloader()

assets = ['AAPL', 'IBM', 'TSLA', 'GOOG', 'NVDA'] 
benchmark = '^GSPC'  
start_date = '2019-01-01'
end_date = '2023-12-31'
rf = .065

asset_prices, benchmark_prices = downloader.download_data(start_date= start_date, end_date= end_date,
                                                          assets= assets, benchmark=benchmark)


# Default limits for every asset (min 1% - max 100%):  boounds = tuple((0.01, 1) for _ in range(self.num_assets))
Asset_allocation = AssetAllocation(asset_prices = asset_prices,benchmark_prices = benchmark_prices, rf=rf)

[*********************100%%**********************]  5 of 5 completed
[*********************100%%**********************]  1 of 1 completed


In [3]:
Asset_allocation.Optimize_Portfolio(n_simulations = 10000) #Default Method == "Montecarlo"

Ticker,AAPL,GOOG,IBM,NVDA,TSLA,Optimized Value
Max Sharpe,0.191213,0.014853,0.049573,0.474559,0.269801,0.079753
Max (Smart) Sharpe,0.059216,0.016758,0.020608,0.554259,0.349159,0.077378
Max Omega,0.022594,0.044646,0.017707,0.897490,0.017563,1.275418
Max (Smart) Omega,0.014139,0.029666,0.042873,0.273602,0.639721,1.126723
Min VaR (Empirical),0.022123,0.098041,0.014549,0.015152,0.850135,0.054244
Min VaR (Parametric),0.095082,0.301062,0.570520,0.016277,0.017058,0.024555
Semivariance,0.101025,0.238451,0.613057,0.021454,0.026013,0.000223
Safety-First,0.218785,0.016000,0.012331,0.485370,0.267514,0.079901
Max Sortino,0.037072,0.013911,0.025496,0.732438,0.191084,0.115445
Risk Parity,0.012938,0.021610,0.056277,0.027653,0.881522,0.001016


In [4]:
Asset_allocation.Optimize_Portfolio(method = "SLSQP")

Ticker,AAPL,GOOG,IBM,NVDA,TSLA,Optimized Value
Max Sharpe,0.221457,0.010000,0.01000,0.501680,0.256863,0.079996
Max (Smart) Sharpe,0.098656,0.010000,0.01000,0.550868,0.330476,0.077521
Max Omega,0.010000,0.010000,0.01000,0.960000,0.010000,1.288639
Max (Smart) Omega,0.010000,0.010000,0.01000,0.255467,0.714533,1.140037
Min VaR (Empirical),0.010000,0.010000,0.01000,0.389210,0.580790,0.049680
Min VaR (Parametric),0.156482,0.217159,0.60636,0.010000,0.010000,0.024424
Semivariance,0.200000,0.200000,0.20000,0.200000,0.200000,0.000401
Safety-First,0.221457,0.010000,0.01000,0.501680,0.256863,0.079996
Max Sortino,0.010000,0.010000,0.01000,0.747532,0.222468,0.115912
Risk Parity,0.200000,0.200000,0.20000,0.200000,0.200000,0.000014


In [5]:
Asset_allocation.Optimize_Portfolio(method = "Genetic", population_size=100, generations=200, crossover_rate=0.7, mutation_rate=0.1)

Ticker,AAPL,GOOG,IBM,NVDA,TSLA,Optimized Value
Max Sharpe,0.063020,0.099974,0.037093,0.522190,0.277723,0.078577
Max (Smart) Sharpe,0.078839,0.078789,0.017281,0.487725,0.337366,0.078725
Max Omega,0.047227,0.027803,0.090642,0.453091,0.381236,1.191447
Max (Smart) Omega,0.134129,0.063536,0.017697,0.218908,0.565731,1.162673
Min VaR (Empirical),0.170293,0.082466,0.021074,0.153216,0.572951,0.047763
Min VaR (Parametric),0.115925,0.283621,0.319120,0.065802,0.215531,0.032033
Semivariance,0.069659,0.039991,0.724484,0.025043,0.140823,0.000255
Safety-First,0.010245,0.059590,0.126141,0.535315,0.268710,0.078513
Max Sortino,0.186708,0.014833,0.102738,0.531837,0.163884,0.112691
Risk Parity,0.011205,0.013080,0.010124,0.019900,0.945691,0.001179


In [6]:
Asset_allocation.Optimize_Portfolio(method = "Gradient", learning_rate=0.01, max_iters=1000, tol=1e-6)

Ticker,AAPL,GOOG,IBM,NVDA,TSLA,Optimized Value
Max Sharpe,0.218456,0.079268,0.141724,0.316454,0.244097,0.077620
Max (Smart) Sharpe,0.201195,0.067792,0.137082,0.314393,0.279538,0.077745
Max Omega,0.205361,0.189996,0.174982,0.229846,0.199815,1.137665
Max (Smart) Omega,0.176572,0.164524,0.162905,0.221437,0.274561,1.138444
Min VaR (Empirical),0.199397,0.201444,0.199150,0.196247,0.203763,0.035712
Min VaR (Parametric),0.255781,0.282329,0.441872,0.010009,0.010009,0.022531
Semivariance,0.200001,0.200002,0.200005,0.199997,0.199995,0.000401
Safety-First,0.218456,0.079268,0.141724,0.316454,0.244097,0.077620
Max Sortino,0.231154,0.029324,0.061046,0.413851,0.264625,0.112007
Risk Parity,0.199999,0.199999,0.199999,0.200001,0.200002,0.000014


-----

-------

--------------

----------